## **Pegasus Encoder Decoder**

Imports

In [1]:
# %pip install accelerate -U
# !pip install transformers[torch]
# !pip install accelerate -U
from src.training_utils import *

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: difra00 (deepl_wizards). Use `wandb login --relogin` to force relogin


In [2]:
import transformers
print(transformers.__version__)

4.30.2


/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load the Pubmed dataset which was downloaded to the local directory 

In [3]:
from datasets import load_dataset
pubmed_dataset = load_dataset("ccdv/pubmed-summarization")#, data_dir ="/home/peppe/Desktop/Università/Projects/SAESum-Simultaneous_Abstractive_and_Extractive_Summarization/src/data/PubMed_abstractive")
print(pubmed_dataset)

No config specified, defaulting to: pubmed-summarization/section
Found cached dataset pubmed-summarization (/home/peppe/.cache/huggingface/datasets/ccdv___pubmed-summarization/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b)
100%|██████████| 3/3 [00:00<00:00, 846.99it/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 119924
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6633
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6658
    })
})


In [4]:
print(pubmed_dataset["train"][1])
print(pubmed_dataset["test"][1])

{'article': 'it occurs in more than 50% of patients and may reach 90% in certain types of cancers , especially in patients undergoing chemotherapy and/or radiation therapy.1 anemia is defined as an inadequate circulating level of hemoglobin ( hb ) ( hb < 12 g / dl ) and may arise as a result of the underlying disease , bleeding , poor nutrition , chemotherapy , or radiation therapy . \n preliminary studies suggest that survival and loco - regional control after radiation therapy , especially in head and neck cancers , may be compromised by anemia.24 anemia often worsens symptoms such as fatigue , weakness , and dyspnea , and thus may have a negative effect on quality of life ( qol ) and performance status in patients with cancer . \n thus , to improve physical functioning , qol , and prognosis in patients with cancer , it would be reasonable to take a proactive approach in identifying populations who need treatment for cancer - associated anemia ( caa ) and provide timely management . 

Visualize some examples

Load tokenizer and model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_checkpoint = "checkpoint-14500-finetuned_alot/checkpoint-2900"#google/pegasus-x-base" # Use pegasus-x-base-finetuned-xsum
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
print(tokenizer)
print(tokenizer(text_target=["Hello, this one sentence!", "This is another sentence."]))

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
print(model)

Load the metric

In [6]:
from evaluate import load
metric = load("rouge")
print(metric)

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

Create pre-processing function

In [7]:
max_input_length = 512
max_target_length = 256
#DA MODIFICARFE PER PEGASUS FORSE
def preprocess_function(examples):
    inputs = [doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["abstract"], max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
preprocess_function(pubmed_dataset['train'][:1])

{'input_ids': [[114, 909, 11624, 1382, 2375, 120, 115, 2651, 110, 108, 47422, 143, 110, 38117, 233, 57468, 110, 158, 604, 404, 3629, 197, 371, 231, 195, 26364, 110, 108, 21472, 132, 11526, 70443, 111, 56744, 143, 13076, 233, 57424, 110, 158, 604, 195, 26364, 110, 108, 21472, 132, 11526, 72225, 115, 109, 1690, 1105, 110, 107, 115, 110, 35057, 114, 692, 790, 110, 58373, 281, 399, 2220, 115, 110, 116, 43449, 111, 35739, 33824, 19244, 2375, 19107, 113, 305, 76040, 110, 108, 110, 74760, 111, 48254, 110, 108, 118, 72225, 110, 108, 15713, 111, 10703, 110, 108, 4802, 110, 107, 109, 19107, 113, 40999, 790, 8225, 399, 4843, 404, 115, 40727, 8032, 6005, 135, 20177, 112, 30316, 110, 107, 53018, 21026, 692, 113, 8225, 399, 392, 115, 32043, 28064, 3264, 120, 30316, 113, 183, 4512, 135, 40999, 111, 580, 513, 1050, 110, 107, 6406, 246, 133, 3251, 233, 4087, 79876, 644, 111, 256, 319, 371, 233, 377, 3957, 113, 3085, 943, 242, 110, 107, 9939, 110, 108, 399, 5173, 962, 127, 850, 130, 109, 1146, 962, 110,

## **TRAINING**

In [8]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer


In [9]:
tokenized_datasets = pubmed_dataset.map(preprocess_function, batched=True)
print(tokenized_datasets["train"][0]["article"])

Loading cached processed dataset at /home/peppe/.cache/huggingface/datasets/ccdv___pubmed-summarization/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b/cache-7644053714deab3a.arrow
Loading cached processed dataset at /home/peppe/.cache/huggingface/datasets/ccdv___pubmed-summarization/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b/cache-5d59a7f99e8ca9db.arrow
Loading cached processed dataset at /home/peppe/.cache/huggingface/datasets/ccdv___pubmed-summarization/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b/cache-538d22dcaed8e414.arrow


a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries . 
 in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively . 
 the prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% . 
 anthropometric study of elementary school students in shiraz revealed that 16% of them suffer from malnutrition and low body weight . 
 snack should have 300 - 400 kcal energy and could provide 5 - 10 g of protein / day . nowadays , school nutrition programs are running as the national programs , world - wide . national school lunch program in the united states 
 there are also some reports regarding school feeding programs in developing countries . in 

In [10]:
batch_size = 2
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    "Pegasus-finetuned_alot",
    evaluation_strategy = "epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate = True,
    fp16=True,
    gradient_accumulation_steps=512,
    logging_steps=1,
    label_smoothing_factor = 0.1, 
    auto_find_batch_size = True, 
    generation_max_length = max_target_length
)

In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    wandb.log({'rouge1': result['rouge1'], 'rouge2': result['rouge2'], 'rougeL': result['rougeL'], 'rougeLsum': result['rougeLsum']})
    

    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [12]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    
)
for param in trainer.model.model.encoder.parameters():
    param.requires_grad = False

In [13]:
trainer.train()
wandb.finish()

/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 1/1170 [00:29<9:32:28, 29.38s/it]

{'loss': 4.4274, 'learning_rate': 0.0004995726495726496, 'epoch': 0.01}


  0%|          | 2/1170 [00:58<9:30:03, 29.28s/it]

{'loss': 4.2296, 'learning_rate': 0.0004991452991452991, 'epoch': 0.02}


  0%|          | 3/1170 [01:27<9:28:36, 29.23s/it]

...

100%|██████████| 1170/1170 [27:19:41<00:00, 29.92s/it]

{'loss': 3.5467, 'learning_rate': 0.0, 'epoch': 9.99}


                                                      
100%|██████████| 1170/1170 [29:22:04<00:00, 90.36s/it]

{'eval_loss': 3.5829203128814697, 'eval_rouge1': 38.6093, 'eval_rouge2': 14.4432, 'eval_rougeL': 22.8834, 'eval_rougeLsum': 34.6125, 'eval_gen_len': 239.5067, 'eval_runtime': 7343.0901, 'eval_samples_per_second': 0.903, 'eval_steps_per_second': 0.452, 'epoch': 9.99}
{'train_runtime': 105724.9824, 'train_samples_per_second': 11.343, 'train_steps_per_second': 0.011, 'train_loss': 3.6156574220738857, 'epoch': 9.99}


eval/gen_len,▁▅█▆▆▆▆▆▆▆
eval/loss,█▅▄▃▂▂▁▁▁▁
eval/rouge1,▂▁▃▂▄▇▆███
eval/rouge2,▃▁▃▃▄▇████
eval/rougeL,▄▁▁▁▄▇▇█▇█
eval/rougeLsum,▁▁▃▃▃▇▇███
eval/runtime,▃▃▄▃▂▃▁▅▇█
eval/samples_per_second,▆▆▅▆▇▆█▄▂▁
eval/steps_per_second,▆▅▅▆▇▆█▄▂▁
rouge1,▂▁▃▂▄▇▆███
rouge2,▃▁▃▃▄▇████


In [35]:
model_checkpoint = "checkpoint-14500-finetuned_alot/checkpoint-29000"

In [36]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [37]:
# import pegasus
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


ARTICLE_TO_SUMMARIZE = "Message Passing Neural Networks (MPNNs) are instances of Graph Neural Networks that \
      leverage the graph to send messages over the edges. This inductive bias leads to a phenomenon known as \
        over-squashing, where a node feature is insensitive to information contained at distant nodes. Despite \
         recent methods introduced to mitigate this issue, an understanding of the causes for over-squashing and \
            of possible solutions are lacking. In this theoretical work, we prove that: (i) Neural network width can mitigate over-squashing, but at the cost of making the whole network more sensitive; (ii) Conversely, depth cannot help mitigate over-squashing: increasing the number of layers leads to over-squashing being dominated by vanishing gradients; (iii) The graph topology plays the greatest role, since over-squashing occurs between nodes at high commute (access) time. Our analysis provides a unified framework to study different recent methods introduced to cope with over-squashing and serves as a justification for a class of methods that fall under graph rewiring."
print(len(ARTICLE_TO_SUMMARIZE))
inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, return_tensors="pt")

# Generate Summary
summary_ids = model.generate(inputs["input_ids"].to(device))
output = tokenizer.batch_decode(summary_ids, skip_special_tokens=True,
                                clean_up_tokenization_spaces=False)[0]
print(output)

print(len(output))

1097


/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (16384) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


we present a unified framework to study different methods introduced to cope with over-squashing and serve as a justification for a class of methods that fall under graph rewiring . our analysis provides a unified framework to study different recent methods introduced to cope with over-squashing and serves as a justification for a class of methods that fall under graph rewiring . in this theoretical work , we prove that: ( i) neural network width can mitigate over-squashing , but at the cost of making the whole network more sensitive ; ( ii) depth cannot help mitigate over-squashing : increasing the number of layers leads to over-squashing being dominated by vanishing gradients ; ( iii) the graph topology plays the greatest role , since over-squashing occurs between nodes at high commute time . our analysis provides a unified framework to study different recent methods introduced to cope with over-squashing and serves as a justification for a class of methods that fall under graph rewi

In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-x-base")

model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-x-base")

In [16]:
input_text = "Studies have shown that owning a dog has numerous benefits."

# Tokenize the input text
input_tokens = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

output_tokens = model.generate(
    input_tokens.input_ids,
    decoder_start_token_id=model.config.pad_token_id,
    max_length=50,  # Set the desired maximum length of the generated output
    num_beams=1,  # Number of beams for beam search
    early_stopping=True,  # Stop generation when all beams have reached the end token
)

# Decode the generated output tokens
decoded_output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print(decoded_output)

India — easy on grounds) air and central and and and and and not air))))))))))))))))))))))))))))))))


In [17]:

# Example input text
input_text = ["Studies have shown that owning a dog has numerous benefits.",
              "hi how are you?"]

# Tokenize the input text
input_tokens = tokenizer(input_text, return_tensors="pt",
                         truncation=True, max_length=100,
                         padding='max_length'
                         )
print(input_tokens['input_ids'].shape)
# Pass the input through the encoder
encoder_outputs = model.model.encoder(**input_tokens)

# # Access the encoder outputs
# encoder_last_hidden_state = encoder_outputs.last_hidden_state
# print(encoder_last_hidden_state.shape)

torch.Size([2, 100])
